# KNN avec modif des données ponderation des variables multiplication par information mutuelle


## Import des librairies

In [1]:
# !pip install geopandas
# !pip install --upgrade matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from datetime import datetime, timezone, timedelta
from sklearn.metrics import confusion_matrix, classification_report
# import geopandas
from sklearn.linear_model import LinearRegression, LogisticRegression

from scipy.stats import chi2_contingency

# Import des librairies bokeh
from bokeh.models import ColumnDataSource, GeoJSONDataSource, HoverTool
from bokeh.plotting import figure, show
from bokeh.models.widgets import Panel, Tabs
from bokeh.layouts import row,column
from bokeh.io import push_notebook,output_notebook, show
output_notebook()  # pour un affichage dans le notebook

Loading BokehJS ...

##Chargement du fichier avec les variables issues de la phase preprocessing

In [2]:
# df_final = pd.read_csv(path, index_col=0)
df_final = pd.read_csv("C:\workspaces\Datascientest\MonProjetMeteo\data\weatherAUS_Kapy_v2_20230708.csv",
                       index_col=0,parse_dates=[1])

In [3]:
# Affichage du nombre de valeurs restantes avec un dropna
df_final.dropna(inplace=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119115 entries, 3 to 145458
Data columns (total 33 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Date                    119115 non-null  datetime64[ns]
 1   Location                119115 non-null  object        
 2   Rainfall                119115 non-null  float64       
 3   WindGustSpeed           119115 non-null  float64       
 4   Temp9am                 119115 non-null  float64       
 5   RainTomorrow            119115 non-null  float64       
 6   DayOfYear               119115 non-null  int64         
 7   Month                   119115 non-null  int64         
 8   Latitude                119115 non-null  float64       
 9   Longitude               119115 non-null  float64       
 10  diffWind3pm9am          119115 non-null  float64       
 11  diffTempMinMax          119115 non-null  float64       
 12  diffPressure9am3pm      119115

##Suppression de quelques variables non utilisées

In [4]:
# suppression du lieu, et des coord géographiques
df = df_final.drop(['Date','Location','Latitude','Longitude'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119115 entries, 3 to 145458
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Rainfall                119115 non-null  float64
 1   WindGustSpeed           119115 non-null  float64
 2   Temp9am                 119115 non-null  float64
 3   RainTomorrow            119115 non-null  float64
 4   DayOfYear               119115 non-null  int64  
 5   Month                   119115 non-null  int64  
 6   diffWind3pm9am          119115 non-null  float64
 7   diffTempMinMax          119115 non-null  float64
 8   diffPressure9am3pm      119115 non-null  float64
 9   Pressure                119115 non-null  float64
 10  DeltaP_1d               119115 non-null  float64
 11  DeltaP_2d               119115 non-null  float64
 12  DeltaP_3d               119115 non-null  float64
 13  diffHimidity9am3pm      119115 non-null  float64
 14  Humidity            

# Jeu de données d'entrainement et de test

In [5]:
# séparation en features et target
target = df["RainTomorrow"]
features = df.drop("RainTomorrow",axis=1)

features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119115 entries, 3 to 145458
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Rainfall                119115 non-null  float64
 1   WindGustSpeed           119115 non-null  float64
 2   Temp9am                 119115 non-null  float64
 3   DayOfYear               119115 non-null  int64  
 4   Month                   119115 non-null  int64  
 5   diffWind3pm9am          119115 non-null  float64
 6   diffTempMinMax          119115 non-null  float64
 7   diffPressure9am3pm      119115 non-null  float64
 8   Pressure                119115 non-null  float64
 9   DeltaP_1d               119115 non-null  float64
 10  DeltaP_2d               119115 non-null  float64
 11  DeltaP_3d               119115 non-null  float64
 12  diffHimidity9am3pm      119115 non-null  float64
 13  Humidity                119115 non-null  float64
 14  DeltaH_1d           

In [6]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import confusion_matrix

In [7]:
# séparation en jeux d'entrainement et de test

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = .2, random_state = 42, shuffle = True)
print(X_train.shape)
print(X_test.shape)
print(pd.DataFrame(y_test).value_counts())

(95292, 28)
(23823, 28)
RainTomorrow
0.0             18590
1.0              5233
dtype: int64


# **Normalisation des données**

Nous allons utiliser un preprocessing de normalisation MinMax.

La normalisation MinMax effectue le calcul suivant sur chaque variable :

X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

X_scaled = X_std * (max - min) + min

Les valeurs des variables sont donc ramenées entre 0 et 1.



In [8]:
from sklearn.preprocessing import MinMaxScaler

# pour plus d'info, voir : https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html

minmax = MinMaxScaler()

X_train_minmax = minmax.fit_transform(X_train,y_train)

X_test_minmax = minmax.transform(X_test)

pd.DataFrame(X_train_minmax).describe().transpose().mean()

count    95292.000000
mean         0.321393
std          0.171141
min          0.000000
25%          0.226067
50%          0.279766
75%          0.409257
max          1.000000
dtype: float64

In [9]:
def Accuracy(TP,TN,FP,FN):
    return (TP+TN)/(TP+FP+FN+TN)

def Precision(TP,FP):
    return TP/(TP+FP)

def Recall(TP,FN):
    return TP/(TP+FN)

def F1score(Recall,Precision):
    return 2*(Recall * Precision) / (Recall + Precision)

def Specificity(TN,FP):
    return TN/(TN+FP)

## simulation

In [10]:
# Première simulation sur le jeu de données normalisées

from sklearn.neighbors import KNeighborsClassifier

In [11]:
from pandas.core.arrays.timedeltas import precision_from_unit

#print(pd.crosstab(y_test,y_predict,rownames=['Realité'],colnames=['Prédiction']))

# TP = pd.crosstab(y_test,y_predict,rownames=['Realité'],colnames=['Prédiction']).iloc[1,1]
# TN = pd.crosstab(y_test,y_predict,rownames=['Realité'],colnames=['Prédiction']).iloc[0,0]
# FP = pd.crosstab(y_test,y_predict,rownames=['Realité'],colnames=['Prédiction']).iloc[0,1]
# FN = pd.crosstab(y_test,y_predict,rownames=['Realité'],colnames=['Prédiction']).iloc[1,0]

# print("\n")
# print("TP",TP)
# print("TN",TN)
# print("FP",FP)
# print("FN",FN)

In [4]:
# print("Accuracy = (TP+TN)/(TP+FP+FN+TN) ==>",Accuracy(TP,TN,FP,FN))
# print("Precision = TP/(TP+FP) ==>",Precision(TP,FP))
# print("Recall = TP/(TP+FN) ==>",Recall(TP,FN))
# print("F1 Score = 2*(Recall * Precision) / (Recall + Precision) ==>",F1score(Recall(TP,FN),Precision(TP,FP)))
# print("Specificity = TN/(TN+FP) ==>",Specificity(TN,FP))

###Information mutuelle

In [12]:
from sklearn.feature_selection import f_regression, mutual_info_regression

f_statistic, p_values = f_regression(X_train_minmax, y_train)
f_tests = f_statistic/max(f_statistic)

# Permet de déterminer sur une dizaine de tirages au sort
# la moyenne de l'information mutuelle
info_mutuelle = []
for i in range(10):
  info_mutuelle.append(mutual_info_regression(X_train_minmax, y_train))

In [13]:
info_mutuelle_moy = np.mean(info_mutuelle,axis=0)

mi_inv = 1/info_mutuelle_moy

data_info = {'Variable':features.columns,'F-test':f_tests,'Info Mutuelle':info_mutuelle_moy, 'mi_inv':mi_inv}
df_info = pd.DataFrame(data=data_info)
print(df_info.sort_values(['mi_inv'], ascending=[True]))
print("On peut voir que les variables ne sont pas classées de la même manière suivant que l'on utilise")
print("F-test ou L'Info Mutuelle")

                  Variable    F-test  Info Mutuelle       mi_inv
13                Humidity  1.000000       0.120967     8.266694
6           diffTempMinMax  0.506554       0.069745    14.337877
0                 Rainfall  0.228365       0.054480    18.355492
17        WindDirInfluence  0.438571       0.048971    20.420259
16               DeltaH_3d  0.367250       0.048531    20.605370
15               DeltaH_2d  0.359039       0.047171    21.199519
14               DeltaH_1d  0.305847       0.044104    22.673647
18  consecutiveRainingDays  0.275921       0.043840    22.810323
12      diffHimidity9am3pm  0.303117       0.038335    26.085930
1            WindGustSpeed  0.229651       0.027204    36.759697
11               DeltaP_3d  0.179028       0.026615    37.572288
8                 Pressure  0.211590       0.025885    38.632843
10               DeltaP_2d  0.143114       0.023286    42.944231
9                DeltaP_1d  0.084158       0.016465    60.735057
7       diffPressure9am3p

###Formule pour un calcul personnalisé de la distance

"Pour les echantillons A et B, sur la dimension i : abs( A(i) - B(i) ) / info mutuelle (i)"

In [14]:
print("Somme de l'info. mutuelle :",sum(info_mutuelle_moy))
print("On obtient 95% de cette information mutuelle à partir de :",sum(info_mutuelle_moy)*0.95)

df_info.sort_values(['mi_inv'], ascending=[True])["Info Mutuelle"].cumsum()

Somme de l'info. mutuelle : 0.6741923002272413
On obtient 95% de cette information mutuelle à partir de : 0.6404826852158793


13    0.120967
6     0.190713
0     0.245192
17    0.294163
16    0.342694
15    0.389865
14    0.433969
18    0.477809
12    0.516144
1     0.543348
11    0.569963
8     0.595848
10    0.619134
9     0.635599
7     0.646904
3     0.652762
23    0.656923
27    0.659910
21    0.662544
20    0.664808
2     0.666911
22    0.668671
24    0.669936
19    0.671073
25    0.672089
4     0.673028
26    0.673898
5     0.674192
Name: Info Mutuelle, dtype: float64

**En regardant le cumul de l'information mutuelle, on obtient 95% de toute l'information mutuelle si l'on garde les variables:**


    13    0.121192 : Humidity
    6     0.191271 : diffTempMinMax
    0     0.247073 : Rainfall
    17    0.294695 : WindDirInfluence
    16    0.341674 : DeltaH_3d
    15    0.388651 : DeltaH_2d
    14    0.431819 : DeltaH_1d
    18    0.474979 : consecutiveRainingDays
    12    0.515267 : diffHimidity9am3pm
    8     0.543139 : Pressure
    1     0.570332 : WindGustSpeed
    11    0.596459 : DeltaP_3d
    10    0.618387 : DeltaP_2d
    9     0.635780 : DeltaP_1d
    7     0.647536 : diffPressure9am3pm


Le jeux d'entrainement et de test filtrés sont donc les suivants :

Création de X_train_minmax_im et de X_test_minmax_im

In [15]:
print(X_train_minmax.shape)
print(X_test_minmax.shape)

(95292, 28)
(23823, 28)


In [16]:
X_train_minmax_im = X_train_minmax*mi_inv

X_test_minmax_im = X_test_minmax*mi_inv

In [17]:
print(X_train_minmax_im.shape)
print(X_test_minmax_im.shape)

(95292, 28)
(23823, 28)


In [18]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


params = {}
params['nb_voisins'] = np.arange(1,20,1)
params['metric'] = ['manhattan']
params['weights'] = ['uniform','distance']

accuracy = []
precision = []
recall = []
f1score = []
specificity = []

CT = []

y_predict = pd.DataFrame()

temps_traitement = []
hdebut = datetime.now()
temps_traitement.append(["Début traitement",hdebut])

for i in params['nb_voisins'] :
    for dist in params['metric']:
        for w in params['weights'] :
            display('calcul en cours : k='+str(i)+'--metric='+str(dist)+'--weights='+str(w),clear = True)
            #initialisation du modèle modèle :
            knn = KNeighborsClassifier(n_neighbors = i, weights = w, algorithm = "auto", leaf_size = 30, p = 2,
                                       metric = dist, metric_params = None, n_jobs = None)
        
            knn.fit(X_train_minmax_im,y_train)
            
            y_p = knn.predict(X_test_minmax_im)
            y_prob = knn.predict_proba(X_test_minmax_im)
            
            col = 'pred'+str(i)+'zzz'+str(dist)+'zzz'+str(w)
            y_predict[col] = y_p
            
            col_proba = 'proba0zzz'+str(i)+'zzz'+str(dist)+'zzz'+str(w)
            y_predict[col_proba] = y_prob[:,0]
            
            col_proba = 'proba1zzz'+str(i)+'zzz'+str(dist)+'zzz'+str(w)
            y_predict[col_proba] = y_prob[:,1]
                             
            ct = pd.crosstab(y_test,y_p,rownames=['Realité'],colnames=['Prédiction'])
            tp = ct.iloc[1,1]
            tn = ct.iloc[0,0]
            fp = ct.iloc[0,1]
            fn = ct.iloc[1,0]
            
            acc = Accuracy(tp,tn,fp,fn)
            pre = Precision(tp,fp)
            rec = Recall(tp,fn)
            f1s = F1score(rec,pre)
            spe = Specificity(tn,fp)
            
            CT.append([i,dist,w,tp,tn,fp,fn,acc,pre,rec,f1s,spe])
                             
            accuracy.append([i,dist,w,acc])
            precision.append([i,dist,w,pre])
            recall.append([i,dist,w,rec])
            f1score.append([i,dist,w,f1s])
            specificity.append([i,dist,w,spe])

hfin = datetime.now()

temps_traitement.append(["Fin traitement",hfin])
temps_traitement.append(["Durée traitement",hfin-hdebut])

warnings.filterwarnings("default", category=FutureWarning)



'calcul en cours : k=19--metric=manhattan--weights=distance'

C:\Users\plebert1\AppData\Local\Temp\ipykernel_8544\2849746911.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  y_predict[col] = y_p
C:\Users\plebert1\AppData\Local\Temp\ipykernel_8544\2849746911.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  y_predict[col_proba] = y_prob[:,0]
C:\Users\plebert1\AppData\Local\Temp\ipykernel_8544\2849746911.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns 

# Sauvegarde des résultats

In [19]:
# Sauvegarde des résultats

#chemin_local = 'C:/workspaces/Datascientest/MonProjetMeteo/data/'

# pond_var_inf_mut_"
chemin_local = ""
nom_fichier = 'accuracy_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=accuracy,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'precision_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=precision,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'recall_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=recall,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'f1score_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=f1score,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'specificity_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=specificity,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'y_predict_pond_var_inf_mut_v1.0.csv'
pd.DataFrame(data=y_predict).to_csv(nom_fichier,sep=',')

nom_fichier = 'crosstab_pond_var_inf_mut_v1.0.csv'
pd.DataFrame(data=CT,
             columns=['k','metric','weights','tp','tn','fp','fn',
                      'acc','pre','rec','f1s','spe']).to_csv(nom_fichier,sep=',')

hsav = datetime.now()
temps_traitement.append(["Fin sauvegarde",hsav])
                         
pd.DataFrame(data=temps_traitement,columns =["Type","Valeur"]).head(10)

Type                      Valeur
0  Début traitement  2023-07-15 19:15:11.586223
1    Fin traitement  2023-07-15 19:20:36.154885
2  Durée traitement              0:05:24.568662
3    Fin sauvegarde  2023-07-15 19:21:03.179664

# Chargement des données

In [20]:
# charger les indicateurs

# from google.colab import drive
# drive.mount('/content/drive')
#chemin_local = 'C:\workspaces\Datascientest\MonProjetMeteo\data\\'

# chemin_local = 'https://github.com/Mediapax/Data/raw/main/'

nom_fichier = 'accuracy_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
accuracy_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'precision_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
precision_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'recall_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
recall_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'f1score_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
f1score_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'specificity_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
specificity_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

# 611+ Mo ==> imposible de charger depuis le compte github :
# "Yowza, that’s a big file. Try again with a file smaller than 25MB."
# nom_fichier = 'y_predict_v1.0.csv'
# y_predict_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'crosstab_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
crosstab_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

accuracy_v1["evaluateur"] = "accuracy_v1"
precision_v1["evaluateur"] = "precision_v1"
recall_v1["evaluateur"] = "recall_v1"
f1score_v1["evaluateur"] = "f1score_v1"
specificity_v1["evaluateur"] = "specificity_v1"

mesures = pd.concat([accuracy_v1,precision_v1,recall_v1,f1score_v1,specificity_v1])

print(accuracy_v1.head(15))
print("\n")

print(precision_v1.head(15))
print("\n")

print(recall_v1.head(15))
print("\n")

print(f1score_v1.head(15))
print("\n")

print(specificity_v1.head(15))
print("\n")

# 611+ Mo ==> impossible de charger depuis le compte github :
# "Yowza, that’s a big file. Try again with a file smaller than 25MB."
# print(y_predict_v1.head(15))
# print("\n")

print(crosstab_v1.head(15))

    k     metric   weights    valeur   evaluateur
0   1  manhattan   uniform  0.775847  accuracy_v1
1   1  manhattan  distance  0.775847  accuracy_v1
2   2  manhattan   uniform  0.801242  accuracy_v1
3   2  manhattan  distance  0.775847  accuracy_v1
4   3  manhattan   uniform  0.800277  accuracy_v1
5   3  manhattan  distance  0.802460  accuracy_v1
6   4  manhattan   uniform  0.803719  accuracy_v1
7   4  manhattan  distance  0.802964  accuracy_v1
8   5  manhattan   uniform  0.804978  accuracy_v1
9   5  manhattan  distance  0.807203  accuracy_v1
10  6  manhattan   uniform  0.804307  accuracy_v1
11  6  manhattan  distance  0.809134  accuracy_v1
12  7  manhattan   uniform  0.806238  accuracy_v1
13  7  manhattan  distance  0.808169  accuracy_v1
14  8  manhattan   uniform  0.805188  accuracy_v1


    k     metric   weights    valeur    evaluateur
0   1  manhattan   uniform  0.487636  precision_v1
1   1  manhattan  distance  0.487636  precision_v1
2   2  manhattan   uniform  0.666444  precisi

# Représentation graphique


In [21]:
print("Affichage des courbes de valeur des indicateurs en fonction du nombre de voisins")

import warnings
from bokeh.util.warnings import BokehUserWarning
# warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.simplefilter(action='ignore', category=BokehUserWarning)
warnings.simplefilter(action = 'ignore',category = UserWarning)



# Code Bokeh sans utiliser de boucle : beaucoup plus rapide et zero lag.

from numpy.lib.utils import source
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool,LassoSelectTool
from bokeh.models.ranges import DataRange1d
from bokeh.colors import RGB
from bokeh.models import LinearColorMapper
from  bokeh.models import PanTool,ResetTool,HoverTool,WheelZoomTool,SaveTool,BoxZoomTool
from bokeh.models import Legend

params = {}
params['nb_voisins'] = np.arange(2,51,1)
params['metric'] = ['manhattan']
# 'haversine' ==> valide seulement en 2D
params['weights'] = ['uniform','distance']
    
mes_couleurs = ['magenta','maroon','mediumaquamarine','mediumblue','mediumorchid','mediumpurple','mediumseagreen',
                'mediumslateblue','mediumspringgreen','mediumturquoise','mediumvioletred','midnightblue','gold',
                'goldenrod','navy','grey','darksalmon','red']
# 9 metrics & 2 weights = 18 curves = 18 colors
# https://docs.bokeh.org/en/2.4.2/docs/reference/colors.html#bokeh-colors-named

# Attention : au moins UNE couleur de plus que le nb courbes !!!
# on utilise la couleur supplémentaire pour colorer le pt quand la souris passe dessus


source1 = ColumnDataSource(accuracy_v1)
source2 = ColumnDataSource(precision_v1)
source3 = ColumnDataSource(recall_v1)
source4 = ColumnDataSource(f1score_v1)
source5 = ColumnDataSource(specificity_v1)

tt1 = [("k", "@k"),("metric", "@metric"),("weights","@weights"),("valeur", "@valeur")]

#tools='hover,xpan,xwheel_zoom,box_zoom,save,reset'
tools=[HoverTool(),BoxZoomTool(), PanTool(),ResetTool()]

##################### Accuracy
p1 = figure(plot_width=800, plot_height=600,x_axis_label='k', y_axis_label='valeur',title="Accuracy",
           toolbar_location = "below",
           tooltips = tt1,tools = tools)

chgt_couleur1 = []
hover1 = []
legend_item1 = []
g1 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(accuracy_v1['metric'].unique()):
    for j,w in enumerate(accuracy_v1['weights'].unique()):
        ma_source = ColumnDataSource(accuracy_v1[(accuracy_v1['metric']==dist) & (accuracy_v1['weights']==w)])
        g1.append(p1.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],
                           line_width=2)) #,legend_label=dist+'--'+w))
        g1.append(p1.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],
                            size=6)) #,legend_label=dist+'--'+w))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur1.append(p1.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7)) #,legend_label=dist+'--'+w))

        hover1.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur1[-1]]))
        legend_item1.append((dist+'--'+w,[g1[-1],g1[-2],chgt_couleur1[-1]]))

p1.legend.visible = False
legend1 = Legend(items=legend_item1,location="center",click_policy="hide")

p1.add_layout(legend1, 'right')

tab1 = Panel(child=p1, title="Accuracy")

##################### Precision
p2 = figure(plot_width=800, plot_height=600,x_axis_label='k', y_axis_label='valeur',title="Precision",
            x_range = p1.x_range,
            toolbar_location = "below",
            tooltips = tt1,tools = tools)

chgt_couleur2 = []
hover2 = []
legend_item2 = []
g2 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(precision_v1['metric'].unique()):
    for j,w in enumerate(precision_v1['weights'].unique()):
        ma_source = ColumnDataSource(precision_v1[(precision_v1['metric']==dist) & (precision_v1['weights']==w)])
        g2.append(p2.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g2.append(p2.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur2.append(p2.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover2.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur2[-1]]))
        legend_item2.append((dist+'--'+w,[g2[-1],g2[-2],chgt_couleur2[-1]]))

p2.legend.visible = False
legend2 = Legend(items=legend_item2,location="center",click_policy="hide")

p2.add_layout(legend2, 'right')

tab2 = Panel(child=p2, title="Precision")

# ##################### Recall
p3 = figure(plot_width=800, plot_height=600,x_axis_label='k', y_axis_label='valeur',title="Recall",
            x_range = p1.x_range,
            toolbar_location = "below",
            tooltips = tt1,tools = tools)

chgt_couleur3 = []
hover3 = []
legend_item3 = []
g3 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(recall_v1['metric'].unique()):
    for j,w in enumerate(recall_v1['weights'].unique()):
        ma_source = ColumnDataSource(recall_v1[(recall_v1['metric']==dist) & (recall_v1['weights']==w)])
        g3.append(p3.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g3.append(p3.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur3.append(p3.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover3.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur3[-1]]))
        legend_item3.append((dist+'--'+w,[g3[-1],g3[-2],chgt_couleur3[-1]]))

p3.legend.visible = False
legend3 = Legend(items=legend_item3,location="center",click_policy="hide")

p3.add_layout(legend3, 'right')

tab3 = Panel(child=p3, title="Recall")

# ##################### F1-Score
p4 = figure(plot_width=800, plot_height=600,x_axis_label='k', y_axis_label='valeur',title="F1-Score",
            x_range = p1.x_range,
            toolbar_location = "below",
           tooltips = tt1,tools = tools)

chgt_couleur4 = []
hover4 = []
legend_item4 = []
g4 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(f1score_v1['metric'].unique()):
    for j,w in enumerate(f1score_v1['weights'].unique()):
        ma_source = ColumnDataSource(f1score_v1[(f1score_v1['metric']==dist) & (f1score_v1['weights']==w)])
        g4.append(p4.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g4.append(p4.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur4.append(p4.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover4.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur4[-1]]))
        legend_item4.append((dist+'--'+w,[g4[-1],g4[-2],chgt_couleur4[-1]]))

p4.legend.visible = False
legend4 = Legend(items=legend_item4,location="center",click_policy="hide")

p4.add_layout(legend4, 'right')

tab4 = Panel(child=p4, title="F1-Score")

# ##################### Specificity
p5 = figure(plot_width=800, plot_height=600,x_axis_label='k', y_axis_label='valeur',title="Specificity",
            x_range = p1.x_range,
            toolbar_location = "below",
           tooltips = tt1,tools = tools)

chgt_couleur5 = []
hover5 = []
legend_item5 = []
g5 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(specificity_v1['metric'].unique()):
    for j,w in enumerate(specificity_v1['weights'].unique()):
        ma_source = ColumnDataSource(specificity_v1[(specificity_v1['metric']==dist) & (specificity_v1['weights']==w)])
        g5.append(p5.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g5.append(p5.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur5.append(p5.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover5.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur5[-1]]))
        legend_item5.append((dist+'--'+w,[g5[-1],g5[-2],chgt_couleur5[-1]]))
        
p5.legend.visible = False
legend5 = Legend(items=legend_item5,location="center",click_policy="hide")

p5.add_layout(legend5, 'right')

tab5 = Panel(child=p5, title="Specificity")

# ##################### Accuracy en fct du F1-Score

p6 = figure(plot_width=800, plot_height=600,x_axis_label='F1-Score', y_axis_label='Accuracy',
#             title="Accuracy en fct du F1-Score",
            toolbar_location = "below",
           x_range = DataRange1d(bounds='auto'),
           y_range = DataRange1d(bounds='auto'))

acc_f1 = accuracy_v1.copy(deep=True)
acc_f1.rename(columns={'valeur':'accuracy'},inplace=True)

acc_f1['f1score'] = f1score_v1['valeur']
acc_f1['precision'] = precision_v1['valeur']
acc_f1['recall'] = recall_v1['valeur']
acc_f1['specificity'] = specificity_v1['valeur']

acc_f1['id_couleur'] = round(17 * (acc_f1['accuracy'] * acc_f1['f1score']**2),0).astype(int)
# couleurs arbitraires pour différencier :
# les meilleurs compromis F1Score/Accuracy
# les compromis moyens
# les mauvais compromis

source6 = ColumnDataSource(acc_f1)

ma_cmap = LinearColorMapper(palette=mes_couleurs, 
                             low = min(acc_f1['id_couleur']), 
                             high = max(acc_f1['id_couleur']))
            
p6.circle(x='f1score',y='accuracy',
          fill_color= {"field":"id_couleur", "transform":ma_cmap},
          line_color=None,
          source = source6,
          size=6)

# chgt de couleur qd la souris passe sur un pt
chgt_couleur6 = p6.circle(x='f1score',y='accuracy',source = source6,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7)

tt2 = [("k", "@k"),("metric", "@metric"),("weights","@weights"),("f1score", "@f1score"),("accuracy","@accuracy"),
      ("recall","@recall"),("precision","@precision"),("specificity","@specificity")]

hover6 = HoverTool(
        tooltips=tt2,
        mode='mouse',
        renderers=[chgt_couleur6])

p6.add_tools(hover6)

tab6 = Panel(child=p6, title="Accuracy en fct du F1-Score")

# ##################### Affichages

# h = column(row(p1,p2),row(p3,p4),row(p5,p6))
# show(h)

tabs = Tabs(tabs=[tab6,tab1,tab2,tab3,tab4,tab5])

show(tabs)

Affichage des courbes de valeur des indicateurs en fonction du nombre de voisins
